# Connect to Elasticsearch

In [1]:
from elasticsearch import Elasticsearch
from dotenv import load_dotenv
import os
from elasticsearch import Elasticsearch

load_dotenv()
 
openai_api_key=os.getenv('OPENAI_API_KEY')
elastic_user=os.getenv('ES_USER')
elastic_password=os.getenv('ES_PASSWORD')
elastic_endpoint=os.getenv("ES_ENDPOINT")


url = f"https://{elastic_user}:{elastic_password}@{elastic_endpoint}:9200"
client = Elasticsearch(url, ca_certs = "./http_ca.crt", verify_certs = True)
 
print(client.info())

{'name': 'liuxgm.local', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'h2QwONxsT4Kt-lTRKmPrhg', 'version': {'number': '8.12.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '1665f706fd9354802c02146c1e6b5c0fbcddfbc9', 'build_date': '2024-01-11T10:05:27.953830042Z', 'build_snapshot': False, 'lucene_version': '9.9.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


# Sample dataset

In [2]:
docs = [
    {
        "text": "A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        "metadata": {"year": 1993, "rating": 7.7, "genre": "science fiction", "director": "Steven Spielberg", "title": "Jurassic Park"},
    },
    {
        "text": "Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        "metadata": {"year": 2010, "director": "Christopher Nolan", "rating": 8.2, "title": "Inception"},
    },
    {
        "text": "A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        "metadata": {"year": 2006, "director": "Satoshi Kon", "rating": 8.6, "title": "Paprika"},
    },
    {
        "text":"A bunch of normal-sized women are supremely wholesome and some men pine after them",
        "metadata":{"year": 2019, "director": "Greta Gerwig", "rating": 8.3, "title": "Little Women"},
    },
    {
        "text":"Toys come alive and have a blast doing so",
        "metadata":{"year": 1995, "genre": "animated", "director": "John Lasseter", "rating": 8.3, "title": "Toy Story"},
    },
    {
        "text":"Three men walk into the Zone, three men walk out of the Zone",
        "metadata":{
            "year": 1979,
            "rating": 9.9,
            "director": "Andrei Tarkovsky",
            "genre": "science fiction",
            "rating": 9.9,
            "title": "Stalker",
        }
    }
]

# Indexing data into Elasticsearch

In [3]:
from elasticsearch import helpers

# create the index
client.indices.create(index="movies_self_query")

operations = [
    {
      "_index": "movies_self_query",
      "_id": i,
      "text": doc["text"],
      "metadata": doc["metadata"]
    } for i, doc in enumerate(docs)
]

# Add the documents to the index directly
response = helpers.bulk(
  client,
  operations,
)

# Setup query retriever

In [4]:
from langchain.vectorstores.elasticsearch import ApproxRetrievalStrategy
from typing import List, Union
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.llms import OpenAI
from langchain.vectorstores.elasticsearch import ElasticsearchStore

# Add details about metadata fields
metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. Can be either 'science fiction' or 'animated'.",
        type="string or list[string]",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

document_content_description = "Brief summary of a movie"

# Set up openAI llm with sampling temperature 0
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

class BM25RetrievalStrategy(ApproxRetrievalStrategy):

    def __init__(
        self
    ):
        pass

    def query(
        self,
        query: Union[str, None],
        filter: List[dict],
        **kwargs,
    ):
                
        if query:
            query_clause = [{
                "multi_match": {
                    "query": query,
                    "fields": ["text"],
                    "fuzziness": "AUTO",
                }
            }]
        else:
            query_clause = []


        bm25_query = {
            "query": {
                "bool": {
                    "filter": filter,
                    "must": query_clause
                }
            },
        }

        print("query", bm25_query)

        return bm25_query


vectorstore = ElasticsearchStore(
  index_name="movies_self_query",
  es_connection=client,
  strategy=BM25RetrievalStrategy()
)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


# BM25 Only Retriever

In [5]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

retriever = SelfQueryRetriever.from_llm(
    llm, 
    vectorstore, 
    document_content_description, 
    metadata_field_info, 
    verbose=True
)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template("""
Use the following context movies that matched the user question. Use the movies below only to answer the user's question.

If you don't know the answer, just say that you don't know, don't try to make up an answer.

----
{context}
----
Question: {question}
Answer:
""")

DOCUMENT_PROMPT = PromptTemplate.from_template("""
---
title: {title}                                                                                   
year: {year}  
director: {director}     
---
""")

def _combine_documents(
    docs, document_prompt=DOCUMENT_PROMPT, document_separator="\n\n"
):
    print("docs:", docs)
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = (_context | LLM_CONTEXT_PROMPT | llm)

chain.invoke("Which director directed movies about dinosaurs that was released after the year 1992 but before 2007?")

query {'query': {'bool': {'filter': [{'bool': {'must': [{'term': {'metadata.genre.keyword': 'science fiction'}}, {'range': {'metadata.year': {'gt': 1992}}}, {'range': {'metadata.year': {'lt': 2007}}}]}}], 'must': [{'multi_match': {'query': 'dinosaur', 'fields': ['text'], 'fuzziness': 'AUTO'}}]}}}
docs: [Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'year': 1993, 'rating': 7.7, 'genre': 'science fiction', 'director': 'Steven Spielberg', 'title': 'Jurassic Park'})]


'Steven Spielberg directed Jurassic Park, which was released in 1993.'

In [6]:
# client.indices.delete(index="movies_self_query")